# Import packages

In [1]:
import os
import src
import src.transforms.aggregate
import pandas as pd
from src import features
from src import database
import yfinance as yf
from config import ROOT_DIR, DB_FILE
from src import extract, transform

# Database

In [5]:
# Create database from csv
database.create_from_csv(
    os.path.join(ROOT_DIR, 'app', 'backend', 'transaction.csv'),
    os.path.join(ROOT_DIR, 'app', 'backend', 'database.db')
)

In [2]:
# List all tables of the database
all_tables = database.list_tables(
    os.path.join(ROOT_DIR, 'app', 'backend', 'database.db')
)
print(all_tables)

['account', 'broker', 'order', 'isin', 'transaction']


# Print dataset

## Only one attribute

In [3]:
#db_file = "/home/hao/repositories/dashboard-listing-securities/app/backend/database.db"
attribute = "isinId"  # For example, if you want to process the 'accountId' attribute
dict_attribute = extract.by_attribute(DB_FILE, attribute)
df_attribute = pd.DataFrame(dict_attribute)
df_attribute.head()

Successfully connected to /home/hao/repositories/dashboard-listing-securities/app/backend/database.db


,transaction_date,isin,name,type,quantity,unit_price,total_price
0,2023-10-16,LU0131510165,Independance et Expansion France Small A,OPVCM,2.0,652.06,1304.12
1,2023-10-16,LU1832174962,Independance et Expansion Europe Small A,OPVCM,15.0,130.30,1954.50
2,2023-10-24,LU1832174962,Independance et Expansion Europe Small A,OPVCM,1.0,126.95,126.95
3,2023-10-24,LU0131510165,Independance et Expansion France Small A,OPVCM,1.0,626.81,626.81
4,2023-11-13,LU1832174962,Independance et Expansion Europe Small A,OPVCM,1.0,131.87,131.87


## All attributes

In [4]:
dict_transactions = extract.all_attribute(DB_FILE)
df_transactions = pd.DataFrame(dict_transactions)
df_transactions.head()

Successfully connected to /home/hao/repositories/dashboard-listing-securities/app/backend/database.db


,transaction_date,isin,isin_name,isin_type,broker_name,broker_country,account_number,account_name,order_type,quantity,unit_price,total_price
0,2023-10-16,LU0131510165,Independance et Expansion France Small A,OPVCM,BourseDirect,FRANCE,508TI00083440250EUR,PEA,BUY,2.0,652.06,1304.12
1,2023-10-16,LU1832174962,Independance et Expansion Europe Small A,OPVCM,BourseDirect,FRANCE,508TI00084026141EUR,PEA-PME,BUY,15.0,130.30,1954.50
2,2023-10-24,LU1832174962,Independance et Expansion Europe Small A,OPVCM,BourseDirect,FRANCE,508TI00084026141EUR,PEA-PME,BUY,1.0,126.95,126.95
3,2023-10-24,LU0131510165,Independance et Expansion France Small A,OPVCM,BourseDirect,FRANCE,508TI00083440250EUR,PEA,BUY,1.0,626.81,626.81
4,2023-11-13,LU1832174962,Independance et Expansion Europe Small A,OPVCM,BourseDirect,FRANCE,508TI00084026141EUR,PEA-PME,BUY,1.0,131.87,131.87


# Features engineering

## Add last date / last price

In [5]:
dict_filter_isin = transform.keep_attribute(dict_transactions, ['isin'])
dict_remove_duplicate = transform.remove_duplicate(dict_filter_isin)
dict_last_price = transform.add_last_price(dict_remove_duplicate)
pd.DataFrame(dict_last_price)

,isin,last_date,last_price
0,LU0131510165,2024-07-24,761.880005
1,LU1832174962,2024-07-24,155.589996
2,LU1832175001,2024-07-24,160.050003
3,US5705351048,2024-07-26,1605.719971
4,LU1964632324,2024-07-24,856.719971
5,US0846707026,2024-07-26,437.269989


## Group by attributes

In [6]:
dict_groupBy = transform.group_by(['isin', 'isin_name'], dict_transactions)
pd.DataFrame(dict_groupBy)

,isin,isin_name,quantity,cost_price,total_price
0,LU0131510165,Independance et Expansion France Small A,9.000000,697.446667,6277.020000
1,LU1832174962,Independance et Expansion Europe Small A,40.000000,139.003500,5560.140000
2,LU1832175001,Independance et Expansion Europe Small I,22.000000,162.532274,3575.710020
3,LU1964632324,Independance et Expansion France Small I,2.000000,914.015000,1828.030000
4,US0846707026,Berkshire Hathaway Inc.,21.666812,364.612668,7899.994132
5,US5705351048,Markel Group Inc.,2.560100,1420.094970,3635.585132


## Merge DataFrame

In [7]:
dict_merged = transform.merge_dictionary(dict_groupBy, dict_last_price)
pd.DataFrame(dict_merged)

,isin,isin_name,quantity,cost_price,total_price,last_date,last_price
0,LU0131510165,Independance et Expansion France Small A,9.000000,697.446667,6277.020000,2024-07-24,761.880005
1,LU1832174962,Independance et Expansion Europe Small A,40.000000,139.003500,5560.140000,2024-07-24,155.589996
2,LU1832175001,Independance et Expansion Europe Small I,22.000000,162.532274,3575.710020,2024-07-24,160.050003
3,LU1964632324,Independance et Expansion France Small I,2.000000,914.015000,1828.030000,2024-07-24,856.719971
4,US0846707026,Berkshire Hathaway Inc.,21.666812,364.612668,7899.994132,2024-07-26,437.269989
5,US5705351048,Markel Group Inc.,2.560100,1420.094970,3635.585132,2024-07-26,1605.719971


## Add cumulative return

In [8]:
dict_cumulative_return = transform.add_cumulative_return(dict_merged)
pd.DataFrame(dict_cumulative_return)

,isin,isin_name,quantity,cost_price,total_price,last_date,last_price,total_last_price,cumulative_return
0,LU0131510165,Independance et Expansion France Small A,9.000000,697.446667,6277.020000,2024-07-24,761.880005,6856.920044,0.092385
1,LU1832174962,Independance et Expansion Europe Small A,40.000000,139.003500,5560.140000,2024-07-24,155.589996,6223.599854,0.119324
2,LU1832175001,Independance et Expansion Europe Small I,22.000000,162.532274,3575.710020,2024-07-24,160.050003,3521.100067,-0.015272
3,LU1964632324,Independance et Expansion France Small I,2.000000,914.015000,1828.030000,2024-07-24,856.719971,1713.439941,-0.062685
4,US0846707026,Berkshire Hathaway Inc.,21.666812,364.612668,7899.994132,2024-07-26,437.269989,9474.246645,0.199273
5,US5705351048,Markel Group Inc.,2.560100,1420.094970,3635.585132,2024-07-26,1605.719971,4110.803697,0.130713
